<script>
    console.log("Hello. You'll see this printed in your browser's DevTools / Console. Feel free to delete this line.");
    document.querySelector('head').innerHTML += '<style>.slides { zoom: 1.0 !important; }</style>';
</script>


# Unit Testing

In this lesson we'll learn about:
* `pytest`
* Using fixtures
* Setuptools integration


Unit tetsing is a process by which a piece of software is broken down into its smallest atomic parts (referred to as "units") and tested on a per-unit basis against known expected behaviours

Benefits:
- Improves code quality beyond first write.
- Aids agile development
 - Adds confidence to future changes
- Makes code more _self documenting_
- Encourages design _before_ code is written
- Testing should be a consideration before you start writing code!
 - Test driven development

Properly executed, unit testing is _not_:
- Proof that an application will work in it's entirety (or in a different environment).
- A replacement for functional or end-to-end testing.
- A replacement for code review.
- A barrier to deployment.

# pytest

- Python's defacto unitesting framework.
- Familiar format to people familiar with `unittest` framework
 - Tests seperated by functions
 - Setup and teardown functions available (see later)
 - `assert` statements used to check and fail on values.
- Tests can be scructured as classes instead if preferred (subclassed from unittest.TestCase).
- Large plugin community

# Project layout for testing
- Unit tests seperated off into as many files as you want, usually roughly grouped by application area or testing type.
- Test files (known as 'modules' in pytest) are auto-discovered by filename, looking for *test.py or test_*.py files within your project.
- two main approaches seen:
 - keeping unit tests within the same directory as your app sourcecode (referred to as 'inline testing' in pytest)
 - seperating off into a tests/ folder at the root of your repo (shown below)
 
```
.
├── setup.cfg          # setup.py and setup.cfg as usual in the root of my project
├── setup.py
├── testing_example    # my application code
│   ├── __init__.py
│   ├── main.py
│   └── person.py
└── tests              # all unit test code
    ├── conftest.py
    ├── test_common_fixtures.py
    ├── test_fixture_factory.py
    ├── test_fixture_factory_finalization.py
    ├── test_fixture_finalization.py
    ├── test_fixture_scoping.py
    ├── test_function_arg_fixtures.py
    ├── test_main.py
    └── test_setup_teardown.py
```

# unit test 'Hello World'
```python
> cat main.py 
def product(numa, numb):
    """ returns the product of two numbers """
    return numa * numb
```

```python
> cat tests/test_main.py 
import pytest
from testing_example import main


def test_product():
    assert main.product(2, 3) == 6

def test_product_invalid_type():
    with pytest.raises(TypeError, match=".*unsupported operand type.*") as excinfo:
        assert main.product(-3, None)
```

```python
> pytest -v tests/test_main.py
========================================================================= test session starts =========================================================================
platform darwin -- Python 3.7.2, pytest-4.4.1, py-1.8.0, pluggy-0.9.0 -- /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example/venv/bin/python3
cachedir: .pytest_cache
rootdir: /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example
collected 2 items

tests/test_main.py::test_product PASSED                                                                                                                         [ 50%]
tests/test_main.py::test_product_invalid_type PASSED                                                                                                            [100%]

====================================================================== 2 passed in 0.01 seconds =======================================================================```

# setup and teardown

```
> cat tests/test_setup_teardown.py 
import pytest
from testing_example import Person

def setup_module(self):
    print("\nMODULE SETUP")

def teardown_module(self):
    print("\nMODULE_TEARDOWN")

def test_something():
    assert 2*3 == 6
```

- `pytest -s` will show stdout from running your tests.

```python
> pytest tests/test_setup_teardown.py  -s 
=================================================================================== test session starts ===================================================================================
platform darwin -- Python 3.7.2, pytest-4.4.1, py-1.8.0, pluggy-0.9.0
rootdir: /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example
collected 1 item                                                                                                                                                                          

tests/test_setup_teardown.py 
MODULE SETUP
.
MODULE_TEARDOWN


================================================================================ 1 passed in 0.01 seconds =================================================================================
```

# fixtures

- fixtures used to carry out setup common across multiple tests.
- Useful when
 - complex setups required between tests
 - shared configuration across tests.
- definied throught the `@fixture` decorator
- Longevity can be scoped on a global, per-module or per-test basis.
- Support context handling for finalization

# fixtures as function args

- Probably the simplest form of fixture used.
- fixture is defined as a function with `@fixture` decorator:

```python
> cat tests/conftest.py 
import pytest

@pytest.fixture()
def common_config():
    return {"foo":123, "bar": ["baz"]}
```
- fixture is then added as a param of unit test, allowing it's access by default:

```python
> cat tests/test_common_fixtures.py 
import pytest

def test_shared_fixtures(common_config):
    assert common_config["foo"] == 123
```

# sharing and overwriting fixtures.
- fixtures can be shared between testing modules by placing in `conftest.py` as shown.
- depending on testing folder structure, fixtures may be overriden e.g.
```bash
tests/
├── conftest.py            # define foo() fixture here
├── test_fixture_examples   
│   ├── conftest.py        # override foo()  here
│   ├── test_common_fixtures.py               # overriden foo() used here
│   ├── test_fixture_factory.py               # overriden foo() used here
│   ├── test_fixture_factory_finalization.py  # overriden foo() used here
│   ├── test_fixture_finalization.py          # overriden foo() used here
│   ├── test_fixture_scoping.py               # overriden foo() used here
│   └── test_function_arg_fixtures.py         # overriden foo() used here
├── test_main.py            # original foo() used here
└── test_setup_teardown.py  # original foo() used here
```

# fixtures: per-class and per-module
```python
@pytest.fixture(scope="session")
def session_scoped():
    print("\nsession_scoped")

@pytest.fixture(scope="module")
def module_scoped():
    print("module_scoped")
```
Fixture scope defined as a param in it's definition:
- fixture will not be destroyed until it leaves it's scope, maintaining it's context.
- useful for long lived fixtures used as 'testing tools' e.g smtp_connection, or for shared config overrides
- BEWARE THAT THE OBJECT IS NOT RECREATED BETWEEN TESTS!
- Don't share a fixture across tests if it's contentext/contents could affect your test e.g class members

```python
> cat tests/test_fixture_examples/test_fixture_scoping.py 
import pytest

@pytest.fixture(scope="session")
def session_scoped():
    print("\nsession_scoped")

@pytest.fixture(scope="module")
def module_scoped():
    print("module_scoped")

@pytest.fixture
def function1(tmpdir):
    print("function1")

@pytest.fixture
def function2(function3):
    print("function2")

@pytest.fixture
def function3():
    print("function3")

def test_foo(function1, module_scoped, function2, session_scoped):
    pass
```

```python
> pytest -s tests/test_fixture_examples/test_fixture_scoping.py 
=================================================================================== test session starts ===================================================================================
platform darwin -- Python 3.7.2, pytest-4.4.1, py-1.8.0, pluggy-0.9.0
rootdir: /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example
collected 1 item                                                                                                                                                                          

tests/test_fixture_examples/test_fixture_scoping.py 
session_scoped
module_scoped
function1
function3
function2
.

================================================================================ 1 passed in 0.04 seconds =================================================================================
```

- Session-scoped fixtures are exectued first, then module-scoped, then function parameters in order.
- Note when function3 is executed, because it is a param of function2.

# fixture finalization
fixtures can handle their own teardown via pythons `yeild`
```python
@pytest.fixture(scope="module")
def smtp_connection():
    import smtplib
    smtp_connection = smtplib.SMTP("smtp.gmail.com", 587, timeout=5)
    print("\nSMTP SETUP")
    yield smtp_connection  # provide the fixture value, code below executes after module scope.
    print("\nSMTP TEARDOWN")
    smtp_connection.close()

def test_foo(smtp_connection):
    assert True 
def test_bar(smtp_connection):
    assert True 
def test_baz(smtp_connection):
    assert True 
```

```python
> pytest -s tests/test_fixture_examples/test_fixture_finalization.py
========================================================================= test session starts =========================================================================
platform darwin -- Python 3.7.2, pytest-4.4.1, py-1.8.0, pluggy-0.9.0
rootdir: /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example
collected 3 items

tests/test_fixture_examples/test_fixture_finalization.py
SMTP SETUP
...
SMTP TEARDOWN
```

# fixture factories
Useful when you need multiple instances of a fixture within a test.
```python 
import pytest

@pytest.fixture
def make_person():
    from testing_example import Person
    def _make_person(name, age):
        person = Person(name=name, age=age)
        return person
    return _make_person

def test_fixture_factory(make_person):
    tom = make_person("Tom", 20)
    dick = make_person("Dick", 30)
    harry = make_person("Harry", 40)
    assert tom.age == 20
    assert dick.age == 30
    assert harry.age == 40
```

# fixture factory finalization
Similarly to earlier, `yeild` can be used for a factory, allowing for the context cleanup of all fixtures created by that factory
```python
import pytest

@pytest.fixture
def make_people():
    from testing_example import Person
    created_people = []

    def _make_person(name, age):
        person = Person(name=name, age=age)
        created_people.append(person)
        return person

    yield _make_person  

    for person in created_people:
        person.die()  # prints a notification to stdout

def test_fixture_factory_teardown(make_people):
    tom = make_people("Tom", 20)
    dick = make_people("Dick", 30)
    harry = make_people("Harry", 40)
    assert tom.age == 20
    assert dick.age == 30
    assert harry.age == 40
    print("\nALL IS WELL, TIME TO DIE")
```

```python
> pytest -s tests/test_fixture_examples/test_fixture_factory_finalization.py
============================================================================================= test session starts ==============================================================================================
platform darwin -- Python 3.7.2, pytest-4.4.1, py-1.8.0, pluggy-0.9.0
rootdir: /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example
collected 1 item

tests/test_fixture_examples/test_fixture_factory_finalization.py
ALL IS WELL, TIME TO DIE

.Tom died at the ripe old age of 20
Dick died at the ripe old age of 30
Harry died at the ripe old age of 40


=========================================================================================== 1 passed in 0.03 seconds ===========================================================================================
```

# paramaterizing fixtures
Takes advantage of builtin `request.param` fixture has access to:
``` python
import pytest
from testing_example import Person

@pytest.fixture(scope="module",
                params=[("tom",10), ("dick",20), ("harry",30)])
def person(request):
    person = Person(name=request.param[0], age=request.param[1])
    yield person
    person.die()


def test_fixture_params(person):
    assert type(person.name) == str
```

```bash
> pytest tests/test_fixture_examples/test_fixture_params.py 
=================================================================================== test session starts ===================================================================================
platform darwin -- Python 3.7.2, pytest-4.4.1, py-1.8.0, pluggy-0.9.0
rootdir: /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example
collected 3 items                                                                                                                                                                         

tests/test_fixture_examples/test_fixture_params.py ...                                                                                                                              [100%]

================================================================================ 3 passed in 0.03 seconds =================================================================================
(venv) mmulhern@C02XF4HMJG5J:testing_example
> pytest -s tests/test_fixture_examples/test_fixture_params.py 
=================================================================================== test session starts ===================================================================================
platform darwin -- Python 3.7.2, pytest-4.4.1, py-1.8.0, pluggy-0.9.0
rootdir: /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example
collected 3 items                                                                                                                                                                         

tests/test_fixture_examples/test_fixture_params.py .tom died at the ripe old age of 10
.dick died at the ripe old age of 20
.harry died at the ripe old age of 30


================================================================================ 3 passed in 0.01 seconds =================================================================================
```

# setuptools integration
Integration with setup.py is trivial:

1) in setup.py:
```python
setup(name='testing_example',
      ...
      setup_requires=['pytest-runner'],  #pytest-runner is a setuptools plugin for running unit tests
      tests_require=['pytest']  # sets pytest as a dependency for 'setup.py test' instead of including in requirements.txt
      )
```
2) (optional) set an alias for 'pytest' to test in setup.cfg:
```bash
> cat setup.cfg
[aliases]
test=pytest
> python setup.py test
running pytest
Searching for pytest
Reading https://pypi.org/simple/pytest/
Downloading https://files.pythonhosted.org/packages/5d/c3/54f607bc9817fd284073ac68e99123f86616f431f9d29a855474b7cf00eb/pytest-4.4.1-py2.py3-none-any.whl#sha256=3773f4c235918987d51daf1db66d51c99fac654c81d6f2f709a046ab446d5e5d
Best match: pytest 4.4.1
Processing pytest-4.4.1-py2.py3-none-any.whl
Installing pytest-4.4.1-py2.py3-none-any.whl to /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example/.eggs
writing requirements to /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example/.eggs/pytest-4.4.1-py3.7.egg/EGG-INFO/requires.txt

Installed /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example/.eggs/pytest-4.4.1-py3.7.egg
running egg_info
writing testing_example.egg-info/PKG-INFO
writing dependency_links to testing_example.egg-info/dependency_links.txt
writing top-level names to testing_example.egg-info/top_level.txt
reading manifest file 'testing_example.egg-info/SOURCES.txt'
writing manifest file 'testing_example.egg-info/SOURCES.txt'
running build_ext
============================================================================================= test session starts ==============================================================================================
platform darwin -- Python 3.7.2, pytest-4.4.1, py-1.8.0, pluggy-0.9.0
rootdir: /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example
collected 12 items

tests/test_main.py ..                                                                                                                                                                                    [ 16%]
tests/test_setup_teardown.py .                                                                                                                                                                           [ 25%]
tests/test_fixture_examples/test_common_fixtures.py .                                                                                                                                                    [ 33%]
tests/test_fixture_examples/test_fixture_factory.py .                                                                                                                                                    [ 41%]
tests/test_fixture_examples/test_fixture_factory_finalization.py .                                                                                                                                       [ 50%]
tests/test_fixture_examples/test_fixture_finalization.py ...                                                                                                                                             [ 75%]
tests/test_fixture_examples/test_fixture_scoping.py .                                                                                                                                                    [ 83%]
tests/test_fixture_examples/test_function_arg_fixtures.py ..                                                                                                                                             [100%]

========================================================================================== 12 passed in 0.38 seconds ===========================================================================================
```

# class based testing
- All tests shown currently have been 'unittest style' function-based definitions.
- Alternatively 'java-style' test classes can be definied via subclassing

```python
import pytest
import unittest

class MyTest(unittest.TestCase):
    @pytest.fixture(autouse=True) # 'autouse' param sets scope to all class methods
    def initdir(self, tmpdir):
        tmpdir.chdir() # change to pytest-provided temporary directory
        tmpdir.join("samplefile.ini").write("# testdata")

    def test_method(self):
        with open("samplefile.ini") as f:
            s = f.read()
        assert "testdata" in s
```

```python
(venv) mmulhern@C02XF4HMJG5J:testing_example
> pytest  -k "test_method"
============================================================================================= test session starts ==============================================================================================
platform darwin -- Python 3.7.2, pytest-4.4.1, py-1.8.0, pluggy-0.9.0
rootdir: /Users/mmulhern/github/BelfastTechTraining/python/06-testing/testing_example
collected 13 items / 12 deselected / 1 selected

tests/test_hello_class.py .                                                                                                                                                                              [100%]

=================================================================================== 1 passed, 12 deselected in 0.05 seconds ====================================================================================
```